# Effective dimension exemple

Implementation test of the [Abbas' et al] paper available on [arXiv](https://arxiv.org/abs/2112.04807). The initial implementation are available here :
- [amyami187/effective_dimension](https://github.com/amyami187/effective_dimension)
- [amyami187/local_effective_dimension](https://github.com/amyami187/local_effective_dimension)

The first objective of the paper is to create a capacity formula that quantify the number of parameters that are truely active in a statistical model that can be **classical** or **quantum**.

## Goal
The main objective of this paper is to create a capacity formula that quantify the number of paramters that are truely active in a statistical model to study the power of QNN (Quantum Neural Networks) over the NN (classical Neural Networks).

Abbas' work is the first capacity measure that can handle :
- generalization bound
- correlation generalization
- scale invariant
- data dependant
- training dependant
- finite data
- efficient evaluation

## Some mathematics explanation and simplification

In all the exaplanation, we have 
- $\Theta \subset \mathbb{R}^d$ is the parameter space ;
- $\theta \in \Theta$ is a vector of parameters ;
- $\kappa _{n,\gamma } \ =\ \frac{\gamma n}{2\pi \ log\ n}$ which is a constant where $n \in \mathbb{N}$, the number of example in training dataset and $\gamma \in (\frac{2\pi\;log\,n}{n}, 1]$ a factor ;
- $F_{ij}(\theta) = \mathbb{E}_{(x,y) \sim p}\Big[ \Big(\frac{\partial}{\partial \theta_i}log\,f(x,y;\theta)\Big) \Big(\frac{\partial}{\partial \theta_j}log\,f(x,y;\theta)\Big) \Big] \in \mathbb{R}^{d \times d}$ is the Fisher information matrix (FIM) of $p(.;.;\theta)$ ;
- $\overline{F}_{ij}( \theta ) \ :=\ d\frac{V_{\Theta }}{\int _{\Theta } tr( F( \theta )) \ d\theta } F_{ij}( \theta ) \in \mathbb{R}^{d \times d}$ is the normalized FIM.

### Definition : global effective dimension
Given a statistical model $\mathcal{M}_{\theta} := \{p(.; .; \theta) : \theta \in \Theta \subset \mathbb{R}^d\}$ with respect to $n \in \mathbb{N}$ and $\gamma \in (\frac{2\pi\;log\,n}{n}, 1]$, the global effective dimension is defined as
$$d_{n,\gamma }(\mathcal{M}_{\theta }) \ :=\ \frac{2}{log\ \kappa _{n,\gamma }} \,log\left(\frac{1}{V_{\Theta }}\int\nolimits _{\Theta }\sqrt{det( id_{d} \ +\ \kappa _{n,\gamma }\overline{F}( \theta ))} \ d\theta \right)$$
where $V_{\theta} := \int_{\Theta}\, d\theta \; \in \mathbb{R}_+$ is the volume of the parameter space.

This definition come from [this paper](https://arxiv.org/abs/2001.10872) and from [this paper](https://www.nature.com/articles/s43588-021-00084-1).


### First simplification
Indeed, we can simplify the formula first by removing the $\sqrt{\;\;}$ using the functions $log$ and $exp$ :
$$\sqrt{det( id_{d} \ +\ \kappa _{n,\gamma }\overline{F}( \theta ))} = exp \Big(\frac{1}{2} log\;det(id_d + \kappa _{n,\gamma }\overline{F}( \theta ))\Big)$$
Then, one can simplify the logarithm of the determinant of the matrix into the trace of the logarithm of the matrix, e.g.
$$exp \Big(\frac{1}{2} log\;det(id_d + \kappa _{n,\gamma }\overline{F}( \theta ))\Big) = exp \Big(\frac{1}{2} tr\;log(id_d + \kappa _{n,\gamma }\overline{F}( \theta ))\Big)$$
Finally, we can use the definition of the trace ($tr(A) = \sum_{i=1}^n a_{ii}$ where $A$ is a square matrix) and so we get this formula:
$$\sqrt{det( id_{d} \ +\ \kappa _{n,\gamma }\overline{F}( \theta ))} = exp \Big(\frac{1}{2}\sum_{i=1}^{d}log\Big(1 + \kappa _{n,\gamma }\lambda_i(\overline{F}( \theta ))\Big)\Big)$$ where $\lambda_i(\overline{F}( \theta ))$ denotes the $i$-th eigenvalue of $\overline{F}( \theta )$.

By replacing in the initial formula, we obtain
$$d_{n,\gamma }(\mathcal{M}_{\theta }) \ :=\ \frac{2}{log\ \kappa _{n,\gamma }}log\left(\frac{1}{V_{\Theta }}\int\nolimits _{\Theta }exp \Big(\frac{1}{2}\sum_{i=1}^{d}log\Big(1 + \kappa _{n,\gamma }\lambda_i(\overline{F}( \theta ))\Big)\Big) \ d\theta \right)$$
where $\lambda_i(\overline{F}( \theta ))$ denotes the $i$-th eigenvalue of $\overline{F}( \theta )$.


### Local effective dimension
We can now define the local effective dimension. Nevertheless, contrary to the paper, I will not define balls containing a set of parameter vectors of radius $\epsilon$. We will use the fact that we perform the computation on $\theta^*$ which is the parameter vector returned at the end of the training of our statistical model. Thus, we have $V_{\Theta} = 1$ and $\int_{\Theta} d\theta = 1$ because we have no $\theta \in \Theta$ other than $\theta^*$ due to optimization.

This gives us
$$d_{n,\gamma }(\mathcal{M}_{\theta }) \ :=\ \frac{2}{log\ \kappa _{n,\gamma }}log\;exp \Big(\frac{1}{2}\sum_{i=1}^{d}log\Big(1 + \kappa _{n,\gamma }\lambda_i(\overline{F}( \theta ))\Big)\Big)$$

And finally, we obtain
$$d_{n,\gamma }(\mathcal{M}_{\theta }) \ :=\ \frac{1}{log\ \kappa _{n,\gamma }}\sum_{i=1}^{d}log\Big(1 + \kappa _{n,\gamma }\lambda_i(\overline{F}( \theta ))\Big)$$ where $\kappa _{n,\gamma } \ =\ \frac{\gamma n}{2\pi \ log\ n}$, $\overline{F}( \theta ) \ =\ d\frac{1}{tr( F( \theta )) } F( \theta )$ and $\lambda_i(\overline{F}( \theta ))$ denotes the $i$-th eigenvalue of $\overline{F}( \theta )$ and $F(\theta)$ the Fisher information matrix defined above.

## Implementation of the function

In [1]:
!pip install nngeometry

In [2]:
from itertools import product
from typing import Sequence, Union

from nngeometry.metrics import FIM
from nngeometry.object import PMatKFAC
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
def get_fisher_matrix_eigenvalues(fisher_information_matrix: FIM) -> np.ndarray:
    s = fisher_information_matrix.generator.layer_collection.numel()
    full_ls = []
    for layer_id, layer in fisher_information_matrix.generator.layer_collection.layers.items():
        a, g = fisher_information_matrix.data[layer_id]
        evals_a, _ = torch.symeig(a)
        evals_a = torch.nan_to_num(evals_a)
        evals_g, _ = torch.symeig(g)
        evals_g = torch.nan_to_num(evals_g)
        full_ls.append([np.absolute(evals_a.detach().cpu().numpy()), np.absolute(evals_g.detach().cpu().numpy())])
    eigs = []
    contract_tuple = lambda t: t[0] * t[1]
    for ls in full_ls:
        eigs += list(map(contract_tuple, product(ls[0], ls[1])))
    return np.array(np.absolute(eigs))

def get_fisher_matrix(model: nn.Module, dataloader: data.DataLoader, model_output_size: int, 
                      device: Union[str, torch.device] = 'cpu', variant: str = 'classif_logits') -> FIM:
    return FIM(model, dataloader, PMatKFAC, model_output_size, device=device, variant=variant)

In [4]:
def effective_dimension(fisher_matrix: FIM, dimensions: int, training_dataset_size: int, gamma: float = 1, normalized: bool = True) -> float:
    c = 2 * np.pi * np.log(training_dataset_size)
    assert gamma <= 1 and gamma > c / training_dataset_size

    kappa = gamma * training_dataset_size / c

    eigs = get_fisher_matrix_eigenvalues(fisher_matrix)
    trace = fisher_matrix.trace().detach().cpu().numpy()
    normalized_fisher_eigs = dimensions * eigs / trace
    
    numerator = np.sum(np.log(1 + kappa * normalized_fisher_eigs))
    ed = numerator / np.log(kappa)
    
    if normalized:
        return ed / dimensions
    return ed


def get_effective_dimension(model: nn.Module, dataloader: data.DataLoader, model_output_size: int, 
                            training_dataset_size: int, device: Union[str, torch.device] = 'cpu', 
                            variant: str = 'classif_logits', gamma: float = 1, normalized: bool = True) -> float:
    d = count_parameters(model)
    fisher = get_fisher_matrix(model, dataloader, model_output_size, device=device, variant=variant)
    ed = effective_dimension(fisher, d, training_dataset_size, gamma=gamma, normalized=normalized)
    return ed


def count_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## Test on a densely connected NN to solve the MNIST dataset classification

### Define the model

In [5]:
class Model(nn.Module):
    def __init__(self, size: Sequence[int], bias=False):
        super(Model, self).__init__()
        self.size = size
        self.layers = nn.ModuleList([
            nn.Linear(self.size[i - 1], self.size[i], bias=bias) for i in range(1, len(self.size))
        ])

    def forward(self, x):
        x = x.view(x.size(0), -1)
        for i in range(len(self.size) - 2):
            x = F.leaky_relu(self.layers[i](x))
        x = self.layers[-1](x)
        return x

### Fetch dataset et construct loaders

In [6]:
VALIDATION_RATIO = 0.9
BATCH_SIZE = 64
LEARNING_RATE = 0.01
IMAGE_SIZE = 28
OUTPUT_SIZE = 10

# get device name
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# pull MNIST datatset and generate loaders
trainset = datasets.MNIST(root="/tmp/data/", train=True, download=True, transform=transforms.ToTensor())
testset = datasets.MNIST(root="/tmp/data/", train=False, download=True, transform=transforms.ToTensor())
n_train_examples = int(len(trainset) * VALIDATION_RATIO)
n_valid_examples = len(trainset) - n_train_examples
train_data, valid_data = data.random_split(trainset, [n_train_examples, n_valid_examples])

trainloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
validloader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE, shuffle=True)
testloader = DataLoader(dataset=testset, batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /tmp/data/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /tmp/data/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /tmp/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /tmp/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/data/MNIST/raw



### Definition of useful function for training and testing

In [7]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc


def train(model, loader, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for data in loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        acc = calculate_accuracy(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)


def evaluate(model, loader, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for data in loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            acc = calculate_accuracy(outputs, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)

### Training

In [8]:
model = Model([IMAGE_SIZE*IMAGE_SIZE, 30, 30, OUTPUT_SIZE]).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [9]:
epochs = 25
best_valid_loss = float('inf')
for epoch in range(epochs):
    train_loss, train_acc = train(model, trainloader, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, validloader, optimizer, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01
	Train Loss: 0.299 | Train Acc: 90.89%
	 Val. Loss: 0.206 |  Val. Acc: 93.87%
Epoch: 02
	Train Loss: 0.174 | Train Acc: 94.80%
	 Val. Loss: 0.185 |  Val. Acc: 94.65%
Epoch: 03
	Train Loss: 0.146 | Train Acc: 95.61%
	 Val. Loss: 0.160 |  Val. Acc: 95.51%
Epoch: 04
	Train Loss: 0.137 | Train Acc: 95.93%
	 Val. Loss: 0.180 |  Val. Acc: 95.21%
Epoch: 05
	Train Loss: 0.125 | Train Acc: 96.29%
	 Val. Loss: 0.170 |  Val. Acc: 95.40%
Epoch: 06
	Train Loss: 0.121 | Train Acc: 96.53%
	 Val. Loss: 0.158 |  Val. Acc: 95.73%
Epoch: 07
	Train Loss: 0.116 | Train Acc: 96.62%
	 Val. Loss: 0.170 |  Val. Acc: 95.50%
Epoch: 08
	Train Loss: 0.110 | Train Acc: 96.85%
	 Val. Loss: 0.171 |  Val. Acc: 95.49%
Epoch: 09
	Train Loss: 0.107 | Train Acc: 96.92%
	 Val. Loss: 0.147 |  Val. Acc: 96.44%
Epoch: 10
	Train Loss: 0.098 | Train Acc: 97.10%
	 Val. Loss: 0.163 |  Val. Acc: 96.03%
Epoch: 11
	Train Loss: 0.098 | Train Acc: 97.16%
	 Val. Loss: 0.182 |  Val. Acc: 95.22%
Epoch: 12
	Train Loss: 0.096 | T

### Loading best seen model and test and test dataset

In [10]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, testloader, optimizer, criterion, device)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.163 | Test Acc: 96.26%


### Compute the effective dimension

In [11]:
ed = get_effective_dimension(model, trainloader, OUTPUT_SIZE, n_train_examples, device=device, normalized=False)
print("Effective dimension :", ed)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Effective dimension : 4472.986261742599


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2499.)
  


In [12]:
normalized_ed = get_effective_dimension(model, trainloader, OUTPUT_SIZE, n_train_examples, device=device, normalized=True)
print("Normalized effective dimension :", normalized_ed)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Normalized effective dimension : 0.18095061830138096
